# CHAPTER 15
# Processing Sequences Using RNNs and CNNs

In [3]:
# Import built-in Python modules
import time
import os
import sys
sys.path.append("..")

# Import third-party modules
import pandas as pd
import tensorflow as tf
import hvplot.pandas
import holoviews as hv
hv.extension('bokeh')

# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.file_management import Logger
from src.file_management.file_handling import *
from src.file_management.directories import MODELS_DIR, DATA_DIR

In [67]:
RIDERSHIP_DIR = DATA_DIR / "ridership" / "CTA_-_Ridership_-_Daily_Boarding_Totals.csv"

df = pd.read_csv(RIDERSHIP_DIR, parse_dates=["service_date"])
df.columns = ["date", "day_type", "bus", "rail", "total"] # shorter names
df = df.sort_values("date").set_index("date")
df = df.drop("total", axis=1) # no need for total, it's just bus + rail
df = df.drop_duplicates() # remove duplicated months (2011-10 and 2014-07)

# Create a new column for the difference between the current day a week before
diff_7 = df[["bus", "rail"]].diff(7)["2019-03":"2019-05"]

df.head()

,day_type,bus,rail
date,,,
2001-01-01,U,297192,126455
2001-01-02,W,780827,501952
2001-01-03,W,824923,536432
2001-01-04,W,870021,550011
2001-01-05,W,890426,557917


In [5]:
curves = df["2019-03":"2019-05"].hvplot(
    x="date",
    y=["bus", "rail"],
    width=800,
    height=400,
    legend="top_left",
    color=["blue", "red"],
) # type: ignore
delayed_curves = (
    df["2019-03":"2019-05"]
    .shift(7)
    .hvplot(
        x="date",
        y=["bus", "rail"],
        width=800,
        height=400,
        legend="top_left",
        line_dash="dashed",
        color=["blue", "red"],
    )
) # type: ignore
diff_curves = diff_7.hvplot(
    x="date",
    y=["bus", "rail"],
    width=800,
    height=400,
    legend="top_left",
    line_width=3,
    color=["blue", "red"],
) # type: ignore

layout = curves * delayed_curves * diff_curves
layout.opts(toolbar=None, active_tools=[], title="CTA Ridership")

:Overlay
   .NdOverlay.I   :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .NdOverlay.II  :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .NdOverlay.III :NdOverlay   [Variable]
      :Curve   [date]   (value)

In [6]:
print("MAE")
print(diff_7.abs().mean(), end="\n\n")

targets = df[["bus", "rail"]]["2019-03":"2019-05"]
print("MAPE")
print((diff_7 / targets).abs().mean())


MAE
bus     43915.608696
rail    42143.271739
dtype: float64

MAPE
bus     0.082938
rail    0.089948
dtype: float64


In [7]:
period = slice("2001", "2019")
df_monthly = df.drop("day_type", axis=1).resample('M').mean() # compute the mean for each month
rolling_average_12_months = df_monthly[period].rolling(window=12).mean()


layout = rolling_average_12_months.hvplot(color = ["blue", "red"]) * df_monthly.hvplot(color = ["blue", "red"], alpha=0.5)

layout.opts(width=800, height=400, title="12-month rolling average", toolbar=None, active_tools=[])

:Overlay
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [date]   (value)

In [8]:
layout = df_monthly.diff(12)[period].hvplot(color = ["blue", "red"], width=800, height=400, title="12-month difference")
hv.Overlay(layout).opts(toolbar=None, active_tools=[])

:Overlay
   .Curve.I  :Curve   [date]   (value)
   .Curve.II :Curve   [date]   (value)

## The ARMA Model Family

Baseline for forecasting (as well as naive forecasting).
### Models
...

### Picking good hyperparameters
**Run a grid search**  
* $p$, $q$, $P$, $Q$: typically $0$ to $2$, sometimes up to $5$ or $6$
* $d$, $D$: typically $0$ or $1$, sometimes $2$

Model with lowest MAE (or use other metric) wins.

In [9]:
from statsmodels.tsa.arima.model import ARIMA

In [10]:
# Predict a single value, 1 day ahead (1 June 2019)
origin, today = "2019-01-01", "2019-05-31"
rail_series = df.loc[origin:today, "rail"].asfreq("D")

model = ARIMA(rail_series,
              order=(1, 0, 0),
              seasonal_order=(0, 1, 1, 7))
model = model.fit()
y_pred = model.forecast()

print(y_pred)

2019-06-01    427758.626282
Freq: D, dtype: float64


In [11]:
# Predict every day in March, April, and May 2019
origin, start_date, end_date = "2019-01-01", "2019-03-01", "2019-05-31"
time_period = pd.date_range(start_date, end_date)
rail_series = df.loc[origin:end_date, "rail"].asfreq("D")
y_preds = []
for today in time_period.shift(-1): # 2019-02-28 to 2019-05-30
    model = ARIMA(rail_series.loc[origin:today], # train on data up to today
                    order=(1, 0, 0),
                    seasonal_order=(0, 1, 1, 7))
    model = model.fit()
    y_pred = model.forecast()[0]
    y_preds.append(y_pred)

y_preds = pd.Series(y_preds, index=time_period, name="rail")
mae = (y_preds - rail_series[time_period]).abs().mean()

print(f"MAE: {mae:.1f}")

MAE: 32040.7


## Preparing the data for ML Models

**Goal:** forecase tomorrow's ridership based on past 8 weeks (56 days).  
**Input:** sequences (1 per day) of 56 values (from time steps $t-55$ to $t$).  
**Output:** single value: forecase for time step $t+1$.

### `timeseries_dataset_from_array`

**EXAMPLE: `.timeseries_dataset_from_array()`**

In [12]:
my_series = [0, 1, 2, 3, 4, 5]
my_dataset = tf.keras.utils.timeseries_dataset_from_array(
    my_series, # sequence from which samples are generated
    targets=my_series[3:], # list of all target values
    sequence_length=3, # length of each sample
    batch_size=2 # return 2 samples at a time
)
list(my_dataset)

[(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[0, 1, 2],
         [1, 2, 3]])>,
  <tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 4])>),
 (<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[2, 3, 4]])>,
  <tf.Tensor: shape=(1,), dtype=int32, numpy=array([5])>)]

**EXAMPLE: `.window()`**

In [13]:
def to_windows(dataset, length):
    return (
        dataset.window(length, shift=1, drop_remainder=True)
        .flat_map(lambda window: window.batch(length))
    )

In [14]:
dataset = (
    to_windows(tf.data.Dataset.range(6), 4) # 3 inputs + 1 target = 4
    .map(lambda window: (window[:-1], window[-1:])) # split into inputs and targets
)

list(dataset.batch(2))

[(<tf.Tensor: shape=(2, 3), dtype=int64, numpy=
  array([[0, 1, 2],
         [1, 2, 3]], dtype=int64)>,
  <tf.Tensor: shape=(2, 1), dtype=int64, numpy=
  array([[3],
         [4]], dtype=int64)>),
 (<tf.Tensor: shape=(1, 3), dtype=int64, numpy=array([[2, 3, 4]], dtype=int64)>,
  <tf.Tensor: shape=(1, 1), dtype=int64, numpy=array([[5]], dtype=int64)>)]

#### Split data

In [15]:
rail_train = df["rail"]["2016-01":"2018-12"] / 1e6
rail_val = df["rail"]["2019-01":"2019-05"] / 1e6
rail_test = df["rail"]["2019-06":] / 1e6

print(rail_train[:5])

date
2016-01-01    0.319835
2016-01-02    0.365509
2016-01-03    0.287661
2016-01-04    0.703185
2016-01-05    0.727716
Name: rail, dtype: float64


In [16]:
seq_length = 56

train_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_train.to_numpy(),
    targets=rail_train[seq_length:],
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True, # different order to the original data
    seed=42
)

val_ds = tf.keras.utils.timeseries_dataset_from_array(
    rail_val.to_numpy(),
    targets=rail_val[seq_length:],
    sequence_length=seq_length,
    batch_size=32,
)

## Forecasting Using a Linear Model

In [81]:
def setup_train(model, train_ds, val_ds, learning_rate):
    tf.random.set_seed(42)
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        monitor="val_mae", patience=50, restore_best_weights=True
    )
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=["mae"])
    history = model.fit(
        train_ds, validation_data=val_ds, epochs=500, callbacks=[early_stopping_cb]
    )
    print(model.evaluate(val_ds))
    return history

In [82]:
model = tf.keras.Sequential([tf.keras.layers.Dense(1, input_shape=[seq_length])])

setup_train(model, train_ds, val_ds, learning_rate=0.02)

Epoch 1/500
33/33 [==============================] - 1s 6ms/step - loss: 0.0446 - mae: 0.2391 - val_loss: 0.0095 - val_mae: 0.1128
Epoch 2/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0078 - mae: 0.0930 - val_loss: 0.0045 - val_mae: 0.0690
Epoch 3/500
33/33 [==============================] - 0s 3ms/step - loss: 0.0063 - mae: 0.0818 - val_loss: 0.0044 - val_mae: 0.0677
Epoch 4/500
33/33 [==============================] - 0s 3ms/step - loss: 0.0059 - mae: 0.0793 - val_loss: 0.0049 - val_mae: 0.0724
Epoch 5/500
33/33 [==============================] - 0s 3ms/step - loss: 0.0056 - mae: 0.0767 - val_loss: 0.0040 - val_mae: 0.0634
Epoch 6/500
33/33 [==============================] - 0s 3ms/step - loss: 0.0061 - mae: 0.0811 - val_loss: 0.0047 - val_mae: 0.0714
Epoch 7/500
33/33 [==============================] - 0s 3ms/step - loss: 0.0052 - mae: 0.0732 - val_loss: 0.0038 - val_mae: 0.0615
Epoch 8/500
33/33 [==============================] - 0s 3ms/step - loss: 0.0051 - m

In [80]:
history.history["val_mae"][-1] * 1e6

27929.49229478836

## Forecasting Using a Simple RNN

**Single Neuron**

In [47]:
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(1, input_shape=[None, 1]) # any length time steps and 1 dimension (univariate time series)
])

history = setup_train(model, train_ds, val_ds)

Epoch 1/500
33/33 [==============================] - 2s 15ms/step - loss: 0.2370 - mae: 0.5239 - val_loss: 0.0733 - val_mae: 0.3404
Epoch 2/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0585 - mae: 0.2910 - val_loss: 0.0513 - val_mae: 0.2688
Epoch 3/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0284 - mae: 0.1723 - val_loss: 0.0168 - val_mae: 0.1309
Epoch 4/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0164 - mae: 0.1615 - val_loss: 0.0155 - val_mae: 0.1407
Epoch 5/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0157 - mae: 0.1562 - val_loss: 0.0155 - val_mae: 0.1358
Epoch 6/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0155 - mae: 0.1540 - val_loss: 0.0150 - val_mae: 0.1372
Epoch 7/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0151 - mae: 0.1533 - val_loss: 0.0148 - val_mae: 0.1356
Epoch 8/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0149 - 

In [32]:
history.history["val_mae"][-1] * 1e6

102814.4508600235

**32 Neurons and dense layer**

In [38]:
univar_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, input_shape=[None, 1]),
    tf.keras.layers.Dense(1) # no activation function by default
])

history = setup_train(univar_model, train_ds, val_ds)

Epoch 1/500
33/33 [==============================] - 1s 17ms/step - loss: 0.0588 - mae: 0.2339 - val_loss: 0.0111 - val_mae: 0.1322
Epoch 2/500
33/33 [==============================] - 0s 11ms/step - loss: 0.0106 - mae: 0.1220 - val_loss: 0.0084 - val_mae: 0.1131
Epoch 3/500
33/33 [==============================] - 0s 11ms/step - loss: 0.0061 - mae: 0.0846 - val_loss: 0.0034 - val_mae: 0.0649
Epoch 4/500
33/33 [==============================] - 0s 10ms/step - loss: 0.0044 - mae: 0.0656 - val_loss: 0.0026 - val_mae: 0.0533
Epoch 5/500
33/33 [==============================] - 0s 10ms/step - loss: 0.0040 - mae: 0.0600 - val_loss: 0.0024 - val_mae: 0.0497
Epoch 6/500
33/33 [==============================] - 0s 11ms/step - loss: 0.0038 - mae: 0.0580 - val_loss: 0.0025 - val_mae: 0.0480
Epoch 7/500
33/33 [==============================] - 0s 10ms/step - loss: 0.0040 - mae: 0.0611 - val_loss: 0.0021 - val_mae: 0.0435
Epoch 8/500
33/33 [==============================] - 0s 9ms/step - loss: 0.0

In [39]:
history.history["val_mae"][-1] * 1e6

36176.13762617111

## Forecasting Using a Deep RNN

In [40]:
deep_model = tf.keras.Sequential([
 tf.keras.layers.SimpleRNN(32, return_sequences=True, input_shape=[None, 1]),
 tf.keras.layers.SimpleRNN(32, return_sequences=True),
 tf.keras.layers.SimpleRNN(32),
 tf.keras.layers.Dense(1)
])

history = setup_train(deep_model, train_ds, val_ds)

Epoch 1/500
33/33 [==============================] - 3s 38ms/step - loss: 0.0619 - mae: 0.2602 - val_loss: 0.0215 - val_mae: 0.1315
Epoch 2/500
33/33 [==============================] - 1s 23ms/step - loss: 0.0158 - mae: 0.1443 - val_loss: 0.0134 - val_mae: 0.1023
Epoch 3/500
33/33 [==============================] - 1s 24ms/step - loss: 0.0123 - mae: 0.1257 - val_loss: 0.0066 - val_mae: 0.0988
Epoch 4/500
33/33 [==============================] - 1s 19ms/step - loss: 0.0083 - mae: 0.0979 - val_loss: 0.0033 - val_mae: 0.0625
Epoch 5/500
33/33 [==============================] - 1s 23ms/step - loss: 0.0057 - mae: 0.0773 - val_loss: 0.0030 - val_mae: 0.0599
Epoch 6/500
33/33 [==============================] - 1s 24ms/step - loss: 0.0053 - mae: 0.0741 - val_loss: 0.0049 - val_mae: 0.0785
Epoch 7/500
33/33 [==============================] - 1s 24ms/step - loss: 0.0064 - mae: 0.0838 - val_loss: 0.0038 - val_mae: 0.0651
Epoch 8/500
33/33 [==============================] - 1s 23ms/step - loss: 0.

In [41]:
history.history["val_mae"][-1] * 1e6

76922.74451255798

## Forecasting Multivariate Time Series

In [49]:
df_mulvar = df[["bus", "rail"]] / 1e6 # use both bus & rail series as input
df_mulvar["next_day_type"] = df["day_type"].shift(-1) # we know tomorrow's type
df_mulvar = pd.get_dummies(df_mulvar) # one-hot encode the day type

mulvar_train = df_mulvar["2016-01":"2018-12"]
mulvar_valid = df_mulvar["2019-01":"2019-05"]
mulvar_test = df_mulvar["2019-06":]


In [50]:
train_mulvar_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_train.astype(float).to_numpy(), # convert to integer data type
    targets=mulvar_train["rail"][seq_length:],
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)
valid_mulvar_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_valid.astype(float).to_numpy(),
    targets=mulvar_valid["rail"][seq_length:],
    sequence_length=seq_length,
    batch_size=32,
)

In [65]:
mulvar_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, input_shape=[None, 5]),
    tf.keras.layers.Dense(1)
])

history = setup_train(mulvar_model, train_mulvar_ds, valid_mulvar_ds)

Epoch 1/500
33/33 [==============================] - 1s 13ms/step - loss: 0.0212 - mae: 0.1548 - val_loss: 0.0036 - val_mae: 0.0706
Epoch 2/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0028 - mae: 0.0565 - val_loss: 0.0021 - val_mae: 0.0523
Epoch 3/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0023 - mae: 0.0502 - val_loss: 0.0015 - val_mae: 0.0443
Epoch 4/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0021 - mae: 0.0487 - val_loss: 0.0016 - val_mae: 0.0459
Epoch 5/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0021 - mae: 0.0483 - val_loss: 0.0014 - val_mae: 0.0426
Epoch 6/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0020 - mae: 0.0471 - val_loss: 0.0017 - val_mae: 0.0469
Epoch 7/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0020 - mae: 0.0472 - val_loss: 0.0014 - val_mae: 0.0438
Epoch 8/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0019 - 

In [55]:
mulvar_model.evaluate(valid_mulvar_ds)

3/3 [==============================] - 0s 11ms/step - loss: 5.4409e-04 - mae: 0.0223


[0.0005440944223664701, 0.022313043475151062]

In [52]:
history.history["val_mae"][-1] * 1e6

25245.97942829132

## Forecast Both Riderships

In [72]:
def setup_train(model, train_ds, val_ds, metrics=["mae"]):
    tf.random.set_seed(42)
    early_stopping_cb = tf.keras.callbacks.EarlyStopping(
        monitor="val_mae", patience=50, restore_best_weights=True
    )
    opt = tf.keras.optimizers.SGD(learning_rate=0.02, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=metrics)
    history = model.fit(
        train_ds, validation_data=val_ds, epochs=500, callbacks=[early_stopping_cb]
    )
    return history

train_multask_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_train.astype(float).to_numpy(), # convert to integer data type
    targets=mulvar_train[["bus", "rail"]][seq_length:],
    sequence_length=seq_length,
    batch_size=32,
    shuffle=True,
    seed=42
)
valid_multask_ds = tf.keras.utils.timeseries_dataset_from_array(
    mulvar_valid.astype(float).to_numpy(),
    targets=mulvar_valid[["bus", "rail"]][seq_length:],
    sequence_length=seq_length,
    batch_size=32,
)

multask_model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, input_shape=[None, 5]),
    tf.keras.layers.Dense(2)
])

history = setup_train(multask_model, train_multask_ds, valid_multask_ds)

print(multask_model.evaluate(valid_multask_ds)) # best model mae

# print(history.history["val_mae"][-1] * 1e6) # latest mae

Epoch 1/500
33/33 [==============================] - 1s 12ms/step - loss: 0.0257 - mae: 0.1509 - val_loss: 0.0029 - val_mae: 0.0568
Epoch 2/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0043 - mae: 0.0630 - val_loss: 0.0023 - val_mae: 0.0514
Epoch 3/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0033 - mae: 0.0536 - val_loss: 0.0012 - val_mae: 0.0355
Epoch 4/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0029 - mae: 0.0520 - val_loss: 0.0020 - val_mae: 0.0505
Epoch 5/500
33/33 [==============================] - 0s 8ms/step - loss: 0.0027 - mae: 0.0495 - val_loss: 0.0012 - val_mae: 0.0369
Epoch 6/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0025 - mae: 0.0484 - val_loss: 0.0012 - val_mae: 0.0364
Epoch 7/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0023 - mae: 0.0471 - val_loss: 0.0012 - val_mae: 0.0378
Epoch 8/500
33/33 [==============================] - 0s 7ms/step - loss: 0.0021 - 

In [73]:
# compute predictions for the validation set for each time series
Y_preds_valid = multask_model.predict(valid_multask_ds)

# compute the mean absolute error for each time series
for idx, name in enumerate(["bus", "rail"]):
    mae = 1e6 * tf.keras.metrics.mean_absolute_error(
        mulvar_valid[name][seq_length:], Y_preds_valid[:, idx])
    print(name, int(mae))

3/3 [==============================] - 0s 4ms/step
bus 27246
rail 24838
